# Using Pyserini BM25 implementation to search queries on TREC-DL 2020

## Setup

### Downloading dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
main_path = '/content/drive/MyDrive/Unicamp-aula-2/'

import os

if not os.path.exists(main_path):
  os.makedirs(main_path)
else:
  print('Diretório já existente')

Diretório já existente


In [ ]:
!wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P collections/msmarco-passage

--2023-03-04 16:35:13--  https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1057717952 (1009M) [application/gzip]
Saving to: ‘collections/msmarco-passage/collectionandqueries.tar.gz’

collectionandquerie 100%[===================>]   1009M  5.98MB/s    in 1m 55s  

2023-03-04 16:37:09 (8.78 MB/s) - ‘collections/msmarco-passage/collectionandqueries.tar.gz’ saved [1057717952/1057717952]



In [ ]:
!tar xvfz collections/msmarco-passage/collectionandqueries.tar.gz -C collections/msmarco-passage

collection.tsv
qrels.dev.small.tsv
qrels.train.tsv
queries.dev.small.tsv
queries.dev.tsv
queries.eval.small.tsv
queries.eval.tsv
queries.train.tsv


### Pyserini installation

In [ ]:
!pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
!conda install -c conda-forge maven

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

# All requested packages already installed.



In [ ]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules

fatal: destination path 'pyserini' already exists and is not an empty directory.


In [ ]:
!cd pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

In [ ]:
!pip install -e pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/pyserini
  Preparing metadata (setup.py) ... done
  Attempting uninstall: pyserini
    Found existing installation: pyserini 0.20.0
    Uninstalling pyserini-0.20.0:
      Successfully uninstalled pyserini-0.20.0
  Running setup.py develop for pyserini


In [ ]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!conda install faiss-cpu -c pytorch

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

# All requested packages already installed.



## Indexing

In [ ]:
!python -m spacy download en_core_web_sm

/usr/local/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!python pyserini/tools/scripts/msmarco/convert_collection_to_jsonl.py \
 --collection-path {main_path}/collections/msmarco-passage/collection.tsv \
 --output-folder collections/msmarco-passage/collection_jsonl

Converting collection...
Converted 0 docs, writing into file 1
Converted 100,000 docs, writing into file 1
Converted 200,000 docs, writing into file 1
Converted 300,000 docs, writing into file 1
Converted 400,000 docs, writing into file 1
Converted 500,000 docs, writing into file 1
Converted 600,000 docs, writing into file 1
Converted 700,000 docs, writing into file 1
Converted 800,000 docs, writing into file 1
Converted 900,000 docs, writing into file 1
Converted 1,000,000 docs, writing into file 2
Converted 1,100,000 docs, writing into file 2
Converted 1,200,000 docs, writing into file 2
Converted 1,300,000 docs, writing into file 2
Converted 1,400,000 docs, writing into file 2
Converted 1,500,000 docs, writing into file 2
Converted 1,600,000 docs, writing into file 2
Converted 1,700,000 docs, writing into file 2
Converted 1,800,000 docs, writing into file 2
Converted 1,900,000 docs, writing into file 2
Converted 2,000,000 docs, writing into file 3
Converted 2,100,000 docs, writing i

In [ ]:
!git clone https://github.com/castorini/anserini.git --recurse-submodules

fatal: destination path 'anserini' already exists and is not an empty directory.


In [ ]:
!mvn -f anserini/pom.xml clean package appassembler:assemble

[INFO] Scanning for projects...
[INFO] 
[INFO] ------------------------< io.anserini:anserini >------------------------
[INFO] Building Anserini 0.20.1-SNAPSHOT
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- clean:3.2.0:clean (default-clean) @ anserini ---
[INFO] Deleting /content/anserini/target
[INFO] 
[INFO] --- jacoco:0.8.7:prepare-agent (default) @ anserini ---
[INFO] argLine set to -javaagent:/root/.m2/repository/org/jacoco/org.jacoco.agent/0.8.7/org.jacoco.agent-0.8.7-runtime.jar=destfile=/content/anserini/target/jacoco.exec
[INFO] 
[INFO] --- resources:3.3.0:resources (default-resources) @ anserini ---
[INFO] Copying 1174 resources
[INFO] 
[INFO] --- compiler:3.8.1:compile (default-compile) @ anserini ---
[INFO] Changes detected - recompiling the module!
[INFO] Compiling 194 source files to /content/anserini/target/classes
[WARNING] system modules path not set in conjunction with -source 11
[WARNING] /con

In [ ]:
!ls anserini/target/

anserini-0.20.1-SNAPSHOT-fatjar.jar   idx-sample-fw1678049894387
anserini-0.20.1-SNAPSHOT.jar	      idx-sample-fw-stored1678049893875
anserini-0.20.1-SNAPSHOT-javadoc.jar  idx-sample-ll
anserini-0.20.1-SNAPSHOT-sources.jar  idx-sample-ll1678049893764
apidocs				      idx-sample-ll1678049893825
appassembler			      idx-sample-ll1678049894265
ast1678049894847		      idx-sample-ll1678049894344
ast1678049894897		      jacoco.exec
ast1678049894940		      javadoc-bundle-options
ast1678049894987		      maven-archiver
classes				      maven-javadoc-plugin-stale-data.txt
generated-sources		      maven-status
generated-test-sources		      site
idx-sample-fw			      surefire-reports
idx-sample-fw1678049893639	      test-classes
idx-sample-fw1678049894307


In [ ]:
!cp anserini/target/anserini-0.20.1-SNAPSHOT-fatjar.jar pyserini/pyserini/resources/jars

In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input collections/msmarco-passage/collection_jsonl \
  --index indexes/lucene-index-msmarco-passage \
  --generator DefaultLuceneDocumentGenerator \
  --threads 9 \
  --storePositions --storeDocvectors --storeRaw

2023-03-05 21:04:59,982 INFO  [main] index.IndexCollection (IndexCollection.java:393) - Setting log level to INFO
2023-03-05 21:04:59,988 INFO  [main] index.IndexCollection (IndexCollection.java:396) - Starting indexer...
2023-03-05 21:04:59,988 INFO  [main] index.IndexCollection (IndexCollection.java:397) - ============ Loading Parameters ============
2023-03-05 21:04:59,990 INFO  [main] index.IndexCollection (IndexCollection.java:398) - DocumentCollection path: collections/msmarco-passage/collection_jsonl
2023-03-05 21:04:59,990 INFO  [main] index.IndexCollection (IndexCollection.java:399) - CollectionClass: JsonCollection
2023-03-05 21:04:59,991 INFO  [main] index.IndexCollection (IndexCollection.java:400) - Generator: DefaultLuceneDocumentGenerator
2023-03-05 21:04:59,991 INFO  [main] index.IndexCollection (IndexCollection.java:401) - Threads: 9
2023-03-05 21:04:59,992 INFO  [main] index.IndexCollection (IndexCollection.java:402) - Language: en
2023-03-05 21:04:59,992 INFO  [main] 

## Searching

OBS.: Thanks to [Marcus Borela](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula1_buscador_simples.ipynb) and [Gustavo Guedes](https://colab.research.google.com/drive/10z86PObSxqbXczZ9pz0T-QurL21oMShf?usp=sharing) tips on the correct eval dataset to be used.

In [ ]:
topics = 'dl20'
qrels_eval = 'dl20-passage'

In [48]:
!python -m pyserini.search.lucene \
  --index indexes/lucene-index-msmarco-passage \
  --topics {topics} \
  --output runs/run.{topics}.bm25tuned.txt \
  --output-format msmarco \
  --hits 100 \
  --bm25 --k1 0.82 --b 0.68

Setting BM25 parameters: k1=0.82, b=0.68
Running dl20 topics, saving to runs/run.dl20.bm25tuned.txt...
100% 200/200 [00:26<00:00,  7.43it/s]


In [49]:
!head runs/run.{topics}.bm25tuned.txt

3505	4711746	1
3505	3859340	2
3505	7207815	3
3505	6834658	4
3505	3829534	5
3505	6834656	6
3505	5022858	7
3505	4409525	8
3505	5779656	9
3505	106527	10


In [50]:
!python pyserini/tools/scripts/msmarco/msmarco_passage_eval.py \
   pyserini/tools/topics-and-qrels/qrels.{qrels_eval}.txt runs/run.{topics}.bm25tuned.txt

#####################
MRR @10: 1.0
QueriesRanked: 200
#####################


In [51]:
!python -m pyserini.eval.convert_msmarco_run_to_trec_run \
   --input runs/run.{topics}.bm25tuned.txt \
   --output runs/run.{topics}.bm25tuned.trec

Done!


In [52]:
!python pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input pyserini/tools/topics-and-qrels/qrels.{qrels_eval}.txt \
   --output collections/msmarco-passage/qrels.{qrels_eval}.trec

Done!


In [53]:
!pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   collections/msmarco-passage/qrels.{qrels_eval}.trec runs/run.{topics}.bm25tuned.trec

map                   	all	0.2695
ndcg_cut_10           	all	0.4876


In [54]:
!head runs/run.{topics}.bm25tuned.trec

3505 Q0 4711746 1 1.0 anserini
3505 Q0 3859340 2 0.5 anserini
3505 Q0 7207815 3 0.3333333333333333 anserini
3505 Q0 6834658 4 0.25 anserini
3505 Q0 3829534 5 0.2 anserini
3505 Q0 6834656 6 0.16666666666666666 anserini
3505 Q0 5022858 7 0.14285714285714285 anserini
3505 Q0 4409525 8 0.125 anserini
3505 Q0 5779656 9 0.1111111111111111 anserini
3505 Q0 106527 10 0.1 anserini


In [56]:
!head collections/msmarco-passage/qrels.{qrels_eval}.trec

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0
